# Вариант 0.1

In [51]:
import pandas as pd
from weather_parser import parser

In [52]:
DEP_NAME = "dep_datetime"
ARR_NAME = "arr_datetime"
INDEX_WEATHER_NAME = "date"
CITY_NAME = "center"
ERROR_CNT = 0
features_mask = ["wind_dir", "wind_speed", "Т(С)", "f(%)", "R(мм)", "R24(мм)"]

In [53]:
# Считываю датасет
dataset = pd.read_excel("../datasets/data.xlsx", index_col=[0])
# Сортирую по дате вылета, чтобы гаарнитровать что последний элемент укажет на самую последнюю дату
dataset = dataset.sort_values(DEP_NAME)
# Получаю список городов в датасете
uniq_centers = dataset["center"].unique()
# Добавляю в датасет новые фичи
for new_feature in features_mask:
    dataset[new_feature] = None

In [54]:
# Функция конверитрует дату и время в отдельные наборы
def get_data_info(idx, col):
    datatime = str(dataset.loc[idx, col])
    date, time = datatime.split()
    return (map(int, date.split('-')), map(int, time.split(":")))

In [55]:
def add_weather_features(idx, weather_data):
    # Получаю час взлета и посадки конкретного дрона 
    hour_dep, _, _ = get_data_info(idx, DEP_NAME)[1]
    hour_arr, _, _ = get_data_info(idx, ARR_NAME)[1]
    # Получаю год, месяц, день конкретного дрона
    y_dep, m_dep, d_dep = get_data_info(idx, DEP_NAME)[0]
    y_arr, m_arr, d_arr = get_data_info(idx, ARR_NAME)[0]
    # функция для доабвлению 0, чтоыб числа от 1-9 записывались как 01 и т.д
    add_zero = lambda num: f"0{num}" if len(str(num)) == 1 else num
    # Формирую диапозон который возьму ид дате о погоде 
    # при этом, так как значения идут сторого с шагом 3, все неподходящие я преобразую так: hour_dep // 3) * 3
    weather_data_idx_dep = f"{y_dep}.{add_zero(d_dep)}.{add_zero(m_dep)} {add_zero((hour_dep // 3) * 3)}"
    weather_data_idx_arr = f"{y_arr}.{add_zero(d_arr)}.{add_zero(m_arr)} {add_zero((hour_arr // 3) * 3)}"
    try:
        # Получаю нужные данные на добавление
        weather_data_for_item = weather_data.loc[weather_data_idx_dep:weather_data_idx_arr] 
        # Не все типы одинаковые, потому проходусь циклом по всем фичам и применяю либо моду для категориальных, лиюо среднее для числовых
        weather_info = weather_data_for_item.dtypes
        for col_name, dtype in zip(weather_info.index, weather_info):
            if dtype == "object":
                dataset.loc[idx, col_name] = weather_data_for_item[col_name].mode().iloc[0]
            else: 
                dataset.loc[idx, col_name] = weather_data_for_item[col_name].mean()
    except:
        global ERROR_CNT
        ERROR_CNT += 1
        print(f"{ERROR_CNT}: Error!")

In [56]:
# Обрабатываю каждый уникальный город
for center in uniq_centers:
    # Получаю только записи нужного города, в которых указаны время взлёта и посадки
    dataset_center_idx = dataset[
        (dataset[CITY_NAME] == center) & 
        (dataset[DEP_NAME].notna() & dataset[ARR_NAME].notna())
    ].index.to_list()
    # Получаю самую первую дату
    y_f, m_f, _ = get_data_info(
        dataset_center_idx[0], 
        DEP_NAME
    )[0]
    # Получаю самую последнюю
    y_l, m_l, _ = get_data_info(
        dataset_center_idx[-1], 
        DEP_NAME
    )[0]
    # Теперь вытаскиваю данные за все нудные года и месяцы, идя от _f до _l
    weather_data = pd.concat(
        [
            parser(year, range(1, m_l + 1) if year == y_l else range(1, 12 + 1), center)
            for year in range(y_f, y_l + 1)
        ]
    ).set_index(INDEX_WEATHER_NAME)
    # Избавляюсь от multiindex
    weather_data.columns = [col[0] for col in weather_data.columns]
    weather_data.index = [idx[0] for idx in weather_data.index]
    # Все фичи кроме первой (которая категориальная) превращаю в числовые
    for feature in features_mask[1:]:
        weather_data[feature] = pd.to_numeric(weather_data[feature], errors="coerce")
    # для каждой записи нахожу... или нет, нкжные данные
    for idx in dataset_center_idx:
        add_weather_features(idx, weather_data[features_mask])

    print(f"{center} modified")

Тюмень modified
1: Error!
2: Error!
Москва modified
Красноярск modified
Магадан modified
Якутск modified
Иркутск modified
3: Error!
4: Error!
5: Error!
6: Error!
7: Error!
8: Error!
9: Error!
10: Error!
11: Error!
12: Error!
13: Error!
14: Error!
15: Error!
16: Error!
17: Error!
18: Error!
19: Error!
20: Error!
21: Error!
22: Error!
23: Error!
24: Error!
25: Error!
26: Error!
27: Error!
28: Error!
29: Error!
30: Error!
31: Error!
32: Error!
33: Error!
34: Error!
35: Error!
36: Error!
37: Error!
38: Error!
39: Error!
40: Error!
41: Error!
42: Error!
43: Error!
44: Error!
45: Error!
46: Error!
47: Error!
48: Error!
49: Error!
50: Error!
51: Error!
52: Error!
53: Error!
54: Error!
55: Error!
56: Error!
57: Error!
58: Error!
59: Error!
60: Error!
61: Error!
62: Error!
63: Error!
64: Error!
65: Error!
66: Error!
67: Error!
68: Error!
69: Error!
70: Error!
71: Error!
72: Error!
73: Error!
74: Error!
75: Error!
76: Error!
77: Error!
78: Error!
79: Error!
80: Error!
81: Error!
82: Error!
83: E

In [58]:
dataset.to_excel("../datasets/data_weather.xlsx")

In [59]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128844 entries, 22983 to 128280
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   center        128844 non-null  object        
 1   flight_id     120031 non-null  float64       
 2   uav_type      128685 non-null  object        
 3   dep_datetime  120351 non-null  datetime64[ns]
 4   arr_datetime  119838 non-null  datetime64[ns]
 5   duration_min  117773 non-null  float64       
 6   takeoff_lat   126403 non-null  float64       
 7   takeoff_lon   126403 non-null  float64       
 8   landing_lat   126398 non-null  float64       
 9   landing_lon   126398 non-null  float64       
 10  wind_dir      112870 non-null  object        
 11  wind_speed    111556 non-null  object        
 12  Т(С)          112870 non-null  object        
 13  f(%)          112870 non-null  object        
 14  R(мм)         45477 non-null   object        
 15  R24(мм)       0 no